# Fine Tuning of a SQL Model

### Inspired by https://huggingface.co/cssupport/t5-small-awesome-text-to-sql

### Datasets:
- https://huggingface.co/datasets/b-mc2/sql-create-context
- https://huggingface.co/datasets/Clinton/Text-to-sql-v1
- https://huggingface.co/datasets/knowrohit07/know_sql
- https://huggingface.co/datasets/gretelai/synthetic_text_to_sql

In [1]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk, concatenate_datasets
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os
os.environ["WANDB_DISABLED"] = "true"


In [2]:
torch.cuda.is_available()

True

In [3]:
model_name='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_name)

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = original_model.to('cuda')

# Load Datasets

In [4]:
from datasets import load_dataset, DatasetDict, interleave_datasets

try:
    # Try loading the existing merged dataset
    dataset = load_from_disk("merged_dataset")
    print("Loaded Merged Dataset")
except:
    # Load the datasets for SCC, TTS, KS, and the new Gretel dataset

    # Load SCC dataset splits
    dataset_scc_train = load_dataset("b-mc2/sql-create-context", split='train[:80%]')
    dataset_scc_test  = load_dataset("b-mc2/sql-create-context", split='train[-20%:-10%]')
    dataset_scc_val   = load_dataset("b-mc2/sql-create-context", split='train[-10%:]')

    # Load TTS dataset splits and preprocess columns
    dataset_tts_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:80%]')
    dataset_tts_train = dataset_tts_train.remove_columns(['source', 'text']).rename_columns(
        {'instruction': 'question', 'input': 'context', 'response': 'answer'})
    dataset_tts_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-20%:-10%]')
    dataset_tts_test  = dataset_tts_test.remove_columns(['source', 'text']).rename_columns(
        {'instruction': 'question', 'input': 'context', 'response': 'answer'})
    dataset_tts_val   = load_dataset("Clinton/Text-to-sql-v1", split='train[-10%:]')
    dataset_tts_val   = dataset_tts_val.remove_columns(['source', 'text']).rename_columns(
        {'instruction': 'question', 'input': 'context', 'response': 'answer'})

    # Load KS dataset splits
    dataset_ks_train  = load_dataset("knowrohit07/know_sql", split='validation[:80%]')
    dataset_ks_test   = load_dataset("knowrohit07/know_sql", split='validation[-20%:-10%]')
    dataset_ks_val    = load_dataset("knowrohit07/know_sql", split='validation[-10%:]')

    # Load the new Gretel dataset splits and preprocess columns
    dataset_gretel_train = load_dataset("gretelai/synthetic_text_to_sql", split='train[:80%]')
    dataset_gretel_val   = load_dataset("gretelai/synthetic_text_to_sql", split='train[-20%:-10%]')
    dataset_gretel_test  = load_dataset("gretelai/synthetic_text_to_sql", split='train[-10%:]')

    # Keep only necessary columns and rename them
    dataset_gretel_train = dataset_gretel_train.rename_columns(
        {'sql_prompt': 'question', 'sql_context': 'context', 'sql': 'answer'})
    dataset_gretel_val = dataset_gretel_val.rename_columns(
        {'sql_prompt': 'question', 'sql_context': 'context', 'sql': 'answer'})
    dataset_gretel_test = dataset_gretel_test.rename_columns(
        {'sql_prompt': 'question', 'sql_context': 'context', 'sql': 'answer'})
    
    # Remove all columns except 'question', 'context', and 'answer'
    dataset_gretel_train = dataset_gretel_train.remove_columns(
        [col for col in dataset_gretel_train.features if col not in ['question', 'context', 'answer']])
    dataset_gretel_val = dataset_gretel_val.remove_columns(
        [col for col in dataset_gretel_val.features if col not in ['question', 'context', 'answer']])
    dataset_gretel_test = dataset_gretel_test.remove_columns(
        [col for col in dataset_gretel_test.features if col not in ['question', 'context', 'answer']])


    # Concatenate datasets to keep all the rows
    dataset = DatasetDict({
        'train': concatenate_datasets([dataset_scc_train, dataset_tts_train, dataset_ks_train, dataset_gretel_train]),
        'test': concatenate_datasets([dataset_scc_test, dataset_tts_test, dataset_ks_test, dataset_gretel_test]),
        'validation': concatenate_datasets([dataset_scc_val, dataset_tts_val, dataset_ks_val, dataset_gretel_val])
    })

    # Save the merged dataset
    dataset.save_to_disk("merged_dataset")
    print("Merged and Saved Dataset")

# Display the dataset structure
print(dataset)


Loaded Merged Dataset
DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 392193
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 49023
    })
    validation: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 49025
    })
})


In [5]:
print(dataset_scc_train.shape, dataset_tts_train.shape, dataset_ks_train.shape, dataset_gretel_train.shape)

NameError: name 'dataset_scc_train' is not defined

In [6]:
dataset['test'][20]

{'answer': 'SELECT shuji_kondo FROM table_name_2 WHERE pepe_michinoku = "sabin (14:43)"',
 'question': 'What is the Shuji Kondo value related to a PEPE Michinoku value of sabin (14:43)?',
 'context': 'CREATE TABLE table_name_2 (shuji_kondo VARCHAR, pepe_michinoku VARCHAR)'}

# Preprocess the Datasets

You need to convert the datasets into explicit instructions for the LLM.

Then preprocess the prompt-response dataset into tokens and pull out their input_ids.

In [7]:
def tokenize_function(example):
    
#     print(len(example["question"]))
    start_prompt = "Tables:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"
  
    data_zip = zip(example['context'], example['question'])
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids
#     print(prompt[0])
#     print()
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.

try:
    tokenized_datasets = load_from_disk("tokenized_datasets")
    print("Loaded Tokenized Dataset")
except:
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.remove_columns(['question', 'context', 'answer'])
    
    tokenized_datasets.save_to_disk("tokenized_datasets")
    print("Tokenized and Saved Dataset")

Loaded Tokenized Dataset


In [8]:
print(tokenized_datasets.keys())
print(tokenized_datasets['train'][0].keys())
print(tokenized_datasets['train'][0]['input_ids'][:10])
print(tokenized_datasets['train'][0]['labels'][:10])
print(tokenized_datasets)

dict_keys(['train', 'test', 'validation'])
dict_keys(['input_ids', 'labels'])
[4398, 7, 10, 205, 4386, 6048, 332, 17098, 819, 41]
[3, 23143, 14196, 2847, 17161, 599, 1935, 61, 21680, 819]
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 392193
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 49023
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 49025
    })
})


In [9]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (392193, 2)
Validation: (49025, 2)
Test: (49023, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 392193
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 49023
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 49025
    })
})


# Test the Model with Zero Shot Inferencing

In [10]:
index = 0

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cuda')

output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_11 (date VARCHAR, away_team VARCHAR)

Question:
On what Date did the Away team essendon play?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT date FROM table_name_11 WHERE away_team = "essendon"

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Question


So pretty poor - aka garbage.

# Perform Full Fine-Tuning

In [11]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
finetuned_model = finetuned_model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
output_dir = f'./sql-training-{str(int(time.time()))}'

training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=5e-3,
        num_train_epochs=2,
        per_device_train_batch_size=16,     # batch size per device during training
        per_device_eval_batch_size=16,      # batch size for evaluation
        weight_decay=0.01,
        logging_steps=50,
        report_to=None,  # This disables reporting to wandb or other integrations
        evaluation_strategy='steps',        # evaluation strategy to adopt during training
        eval_steps=500,                     # number of steps between evaluation
)

trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
)
    
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


  0%|          | 0/49026 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


KeyboardInterrupt: 

### 2 Epochs

#### 5e-3

Time Taken      = 2h 49m 1s on a laptop with a GeForce RTX 3070 GPU

Training Loss   = 0.023100

Validation Loss = 0.013285

In [13]:
try:
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_1_epoch")
    finetuned_model = finetuned_model.to('cuda')
    to_train = False

except:
    to_train = True
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
    finetuned_model = finetuned_model.to('cuda')
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
%%time

if to_train:
    output_dir = f'./sql-training-{str(int(time.time()))}'

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=5e-3,
        num_train_epochs=2,
        per_device_train_batch_size=16,     # batch size per device during training
        per_device_eval_batch_size=16,      # batch size for evaluation
        weight_decay=0.01,
        logging_steps=50,
        evaluation_strategy='steps',        # evaluation strategy to adopt during training
        eval_steps=500,                     # number of steps between evaluation
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
    )
    
    trainer.train()
    
    finetuned_model.save_pretrained("finetuned_model_2_epoch")

In [15]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_1_epoch")
finetuned_model = finetuned_model.to('cuda')

# Test the Fine Tuned Model with Zero Shot Inferencing

In [22]:
index = 27171
# index = len(dataset['test'])-200

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cuda')

output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_11177 (
    "Date" text,
    "Venue" text,
    "Score" text,
    "Result" text,
    "Competition" text
)

Question:
What is the Competition with a Date with 6 december 2011?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT "Competition" FROM table_11177 WHERE "Date" = '6 december 2011'

---------------------------------------------------------------------------------------------------
FINE-TUNED MODEL - ZERO SHOT:
SELECT "Competition" FROM table_11177 WHERE "Date" = '6 december 2011'


# Evaluate the Model Quantitatively (with ROUGE Metric)

In [ ]:
# Perform inferences for test dataset. Do 25 only, due to time it takes.

questions = dataset['test'][0:25]['question']
contexts = dataset['test'][0:25]['context']
human_baseline_answers = dataset['test'][0:25]['answer']

original_model_answers = []
finetuned_model_answers = []

for idx, question in enumerate(questions):
    
    prompt = f"""Tables:
{contexts[idx]}

Question:
{question}

Answer:
"""
      
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda')

    human_baseline_text_output = human_baseline_answers[idx]
    
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_answers.append(original_model_text_output)
    
    finetuned_model_outputs = finetuned_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))
    finetuned_model_text_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)
    finetuned_model_answers.append(finetuned_model_text_output)

zipped_summaries = list(zip(human_baseline_answers, original_model_answers, finetuned_model_answers))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_answers', 'original_model_answers', 'finetuned_model_answers'])
# df

Compute ROUGE score for this subset of the data.

In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_answers,
    references=human_baseline_answers[0:len(original_model_answers)],
    use_aggregator=True,
    use_stemmer=True,
)
print('ORIGINAL MODEL:')
print(original_model_results)


finetuned_model_results = rouge.compute(
    predictions=finetuned_model_answers,
    references=human_baseline_answers[0:len(finetuned_model_answers)],
    use_aggregator=True,
    use_stemmer=True,
)
print('FINE-TUNED MODEL:')
print(finetuned_model_results)